In [1]:
import itertools

from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [2]:
cbt_dataset = load_dataset('cbt','CN')

Found cached dataset cbt (C:/Users/34941/.cache/huggingface/datasets/cbt/CN/1.1.0/dc4451a8a4b50cebb78fdb19fa9f964b27fcdcef915467b8b7055a3a8d8cef7b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
train_set = cbt_dataset['train']
val_set = cbt_dataset['validation']
test_set = cbt_dataset['test']

In [4]:
train_set

Dataset({
    features: ['sentences', 'question', 'answer', 'options'],
    num_rows: 120769
})

In [5]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [6]:
from tqdm import tqdm

In [7]:
len(train_set)

120769

task: write a preprocess function s.t. for an input: context, question, answer, options, output a format that we can put into data loader:
做成input_id, labels, 然后把input_id和labels都padding成一样长就行了应该。

In [8]:
train_set_lst = []
label_lst = []
padding = -100
max_length = 1024
for i in tqdm(range(len(train_set))):
    example = train_set[i]
    context, question, answer, options = example['sentences'], example['question'], example['answer'], example['options']
    input_text = f"{context} {question} {' '.join(options)}"
    input_ids = tokenizer.encode(input_text, add_special_tokens=True)
    labels = [0]*len(input_ids)
    answer_id = tokenizer.encode(answer)[0]
    labels[-len(options) + options.index(answer)] = answer_id
    if len(input_ids)<=1024:
        padding_length = max_length-len(input_ids)
        input_ids+=[padding]*padding_length
        labels+=[padding]*padding_length
        train_set_lst.append(torch.tensor(input_ids))
        label_lst.append(torch.tensor(labels))


100%|██████████| 120769/120769 [04:44<00:00, 425.19it/s]


In [14]:
dataset = torch.utils.data.TensorDataset(torch.tensor(input_ids,dtype=torch.long),torch.tensor(labels,dtype=torch.long))
train_dataloader = DataLoader(dataset,batch_size=16,shuffle=True)
print('finish')

finish


In [16]:

# Define model and optimizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
optimizer = AdamW(model.parameters(), lr=2e-5)

# Define training loop
def train(model, train_dataloader, optimizer, scheduler=None, num_epochs=1):
    model.train()
    for epoch in tqdm(range(num_epochs)):
        total_loss = 0
        for input_ids, labels in train_dataloader:

            input_ids, labels = input_ids.to(device), labels.to(device)
            outputs = model(input_ids, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        print(f"Epoch {epoch+1} Loss: {total_loss/len(train_dataloader)}")

# Load dataset and create dataloader


# Fine-tune model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
num_training_steps = len(train_dataloader) * 3
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)
train(model, train_dataloader, optimizer, scheduler=scheduler, num_epochs=3)

C:\Users\34941\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 33%|███▎      | 1/3 [00:39<01:19, 39.66s/it]

Epoch 1 Loss: 2.310341170853159


 67%|██████▋   | 2/3 [01:21<00:40, 40.69s/it]

Epoch 2 Loss: 0.0857707670268913


100%|██████████| 3/3 [02:01<00:00, 40.59s/it]

Epoch 3 Loss: 0.13959167774818423
